## **Reminder:**

### For each notebook, you should submit your notebook with all the cells executed, from start to finish, in a fresh run (i.e., first cell number should be [1], second [2], etc.). You can achieve this by selecting Kernel -> Restart and run all.

### **Any deviation from the submission instructions above will result in a mark of zero for the assessment or question part.**

# Part 2: NOTEBOOK: KNOWLEDGE EXTRACTION [29 marks]

In [1]:
!pip install pymysql
import pymysql
import numpy as np
import pandas as pd
!pip install spacy
!pip install scipy
import spacy
import scipy
from datetime import datetime

1 - Importing the data [2 marks]

Using pandas, import the data into a dataframe `df` and remove any data rows that have missing values for the column `author_created`.


In [2]:
# your code here
df = pd.read_csv("data_portfolio_24.csv",encoding='utf-8')
print(df.head())

             author                                       comment_body  \
0            arne-b  Campenaerts actually prepared pretty hard for ...   
1  Noble_Flatulence  The bikewrench subreddit be like "frame slight...   
2             tkcom  That fan should be fined based on the damage c...   
3            Cog348  Chris Froome getting cheered by a French crowd...   
4    Sead_KolaSagan  Thomas escaped the crash that took down everyb...   

   comment_score  comment_award comment_mod_distinction  comment_created  \
0            106              0                     NaN     1.624704e+09   
1             78              0                     NaN     1.624717e+09   
2             79              0                     NaN     1.624717e+09   
3             77              0                     NaN     1.624721e+09   
4             67              0                     NaN     1.624717e+09   

   author_comment_karma  author_post_karma  author_created subreddit  \
0                 54881   

In [3]:
df["author"] = df['author'].fillna("anonymous")
df["comment_mod_distinction"] = df['comment_mod_distinction'].fillna("-")
df["author_created"] = df['author_created'].fillna(0)
df["post_body"] = df["post_body"].astype(str)

In [4]:
df.isnull().sum()

author                     0
comment_body               0
comment_score              0
comment_award              0
comment_mod_distinction    0
comment_created            0
author_comment_karma       0
author_post_karma          0
author_created             0
subreddit                  0
post_title                 0
post_score                 0
post_num_comments          0
post_body                  0
post_created               0
post_award                 0
dtype: int64

2 – Timestamps [2 marks]

Change the timestamp format all columns with date information (`author_created`, `comment_created`, `post_created`) to the format `YYYY-MM-DD HH:mm:ss` (i.e. 2024-05-09 09:30:00) and convert this UTC timezone timestamp to the French/CEST timezone stamp (Hint: Metropolitan France is UTC+2 in summer time)


In [5]:
# your code here
df['author_created'] = pd.to_datetime(df['author_created'], unit='s')
df['comment_created'] = pd.to_datetime(df['comment_created'], unit='s')
df['post_created'] = pd.to_datetime(df['post_created'], unit='s')

In [6]:
df[['author_created', 'comment_created', 'post_created']].head()

,author_created,comment_created,post_created
0,2014-01-26 16:34:12,2021-06-26 10:47:04,2021-06-26 08:00:19
1,2013-01-10 03:29:48,2021-06-26 14:15:01,2021-06-26 08:00:19
2,2010-11-05 18:37:48,2021-06-26 14:15:07,2021-06-26 08:00:19
3,2019-01-14 21:43:15,2021-06-26 15:17:15,2021-06-26 08:00:19
4,2017-09-23 12:08:23,2021-06-26 14:15:22,2021-06-26 08:00:19


3 - Handling data irregularities [4 marks]

Karma and score variables (`author_comment_karma`, `author_post_karma`, and `post_score`) may have inconsistencies (i.e. `post_score` should have the same value for each instance where `post_title` is the same, and `author_comment_karma` and `author_post_karma` should be the same for the same author). Fix these inconsistencies while commenting your choices. Use appropriate print statements to show that this has been achieved.


In [7]:
post_score_map = {}
for _,entry in df.iterrows():
    post_title = entry["post_title"]
    post_score = entry["post_score"]

    if post_title not in post_score_map:
        post_score_map[post_title] = post_score
    else:
        # Update the value if it's inconsistent
        if post_score_map[post_title] != post_score:
            entry["post_score"] = post_score_map[post_title]
    df.at[_,'post_score'] = entry["post_score"]

In [8]:
# your code here
df_title = df.groupby('post_title').filter(lambda g: len(g) > 1).drop_duplicates(subset=['post_title', 'post_score'], keep="first")
df_title[["post_title","post_score"]]

,post_title,post_score
0,[Race Thread] 2021 Tour de France - Stage 1 (2...,159
399,[Race Thread] 2021 Tour de France - Stage 2 (2...,100
740,[Race Thread] 2021 Tour de France - Stage 3 (2...,93
1099,[Race Thread] 2021 Tour de France - Stage 4 (2...,96
1664,[Race Thread] 2021 Tour de France - Stage 5 (2...,85
2434,[Race Thread] 2021 Tour de France - Stage 6 (2...,56
2910,[Race Thread] 2021 Tour de France - Stage 7 (2...,138
3358,[Race Thread] 2021 Tour de France - Stage 8 (2...,111
3991,[Race Thread] 2021 Tour de France - Stage 9 (2...,80
4619,[Race Thread] 2021 Tour de France - Stage 10 (...,66


4 - New descriptive columns [2 marks]

Create the following new columns:

a - `comment_length`: calculated by counting the number of characters in comment_body [0.5 mark]

b - `author_total_karma`: summing `author_comment_karma` and `author_post_karma` [0.5 mark]

c - `higher_post_karma`: which has a value of 1 if `author_post_karma` is strictly higher than `author_comment_karma` and has a value of 0 otherwise. [1 mark]


In [9]:
# your code here
df['comment_length'] = df['comment_body'].apply(len)
df['author_total_karma'] = df['author_comment_karma'] + df['author_post_karma']
df['higher_post_karma'] = (df['author_post_karma'] > df['author_comment_karma']).astype(int)

In [10]:
df[['comment_length', 'author_total_karma', 'higher_post_karma']].head()

,comment_length,author_total_karma,higher_post_karma
0,129,66053,0
1,86,186751,0
2,56,88488,0
3,151,95836,0
4,73,32238,0


In [11]:
df['post_title']

0        [Race Thread] 2021 Tour de France - Stage 1 (2...
1        [Race Thread] 2021 Tour de France - Stage 1 (2...
2        [Race Thread] 2021 Tour de France - Stage 1 (2...
3        [Race Thread] 2021 Tour de France - Stage 1 (2...
4        [Race Thread] 2021 Tour de France - Stage 1 (2...
                               ...                        
11805    [Race Thread] 2021 Tour de France - Stage 21 (...
11806    [Race Thread] 2021 Tour de France - Stage 21 (...
11807    [Race Thread] 2021 Tour de France - Stage 21 (...
11808    [Race Thread] 2021 Tour de France - Stage 21 (...
11809    [Race Thread] 2021 Tour de France - Stage 21 (...
Name: post_title, Length: 11810, dtype: object

5 - Extracting information: stage number [2 marks]

Create a new column `stage_nbr` that extracts the stage number as an integer from `post_title`.


In [12]:
# your code here
# df["stage_nbr"] = df["post_title"].str.extract("(\d*\.?\d+)", expand=True)
df['stage_nbr'] = df.post_title.str.split('\Stage').str[1]
df['stage_nbr'] = df['stage_nbr'].str[1:2]
df['stage_nbr'] = df['stage_nbr'].astype(int)

In [13]:
df['stage_nbr'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

6 - Extracting information: stage type [3 marks]

Create a new column `stage_type` that extracts the stage type as a string from `post_body`. [2 marks]

Print out the stage number and stage type for all stages. [1 mark]

In [14]:
# your code here
df['stage_type'] = df['post_body'].str.extract('km\|\[(.*?)\]')

In [15]:
df['stage_type']

0        Medium
1        Medium
2        Medium
3        Medium
4        Medium
          ...  
11805      Easy
11806      Easy
11807      Easy
11808      Easy
11809      Easy
Name: stage_type, Length: 11810, dtype: object

7 - Stage 11 [2 marks]

Create a new dataframe s11 that only includes comments that were posted on the day (in CEST) of stage 11 (7 July 2021) on the post of the 11th stage of the competition. Sort the rows from oldest comment to newest comment.


In [16]:
# your code here
df_stage_11 = df[df.post_title.str.contains('Stage 11')]
s11 = df_stage_11[df_stage_11['comment_created'].dt.date == datetime.strptime('21-07-07', '%y-%m-%d').date()]
s11 = s11.sort_values(by='comment_created')
s11 = s11.reset_index(drop=True)

In [17]:
s11.head()

,author,comment_body,comment_score,comment_award,comment_mod_distinction,comment_created,author_comment_karma,author_post_karma,author_created,subreddit,...,post_score,post_num_comments,post_body,post_created,post_award,comment_length,author_total_karma,higher_post_karma,stage_nbr,stage_type
0,GwenTheChonkster,Only two times up Mont Ventoux? Cycling has de...,43,0,-,2021-07-07 08:03:34,9628,517,2021-06-29 10:44:41,peloton,...,134,4091,| Date | Stage | From > To | Length | Type | F...,2021-07-07 08:00:20,0,69,10145,0,1,Hard
1,GwenTheChonkster,Only two times up Mont Ventoux? Cycling has de...,44,0,-,2021-07-07 08:03:34,9628,517,2021-06-29 10:44:41,peloton,...,134,4091,| Date | Stage | From > To | Length | Type | F...,2021-07-07 08:00:20,0,69,10145,0,1,Hard
2,nz-is-beautiful,"Just tuned in, has ~~MVDP attacked~~ Pogacar f...",22,0,-,2021-07-07 08:05:54,15137,15352,2013-05-10 04:18:57,peloton,...,134,4091,| Date | Stage | From > To | Length | Type | F...,2021-07-07 08:00:20,0,68,30489,1,1,Hard
3,anonymous,A couple of links ahead of the stage:\n\n* If ...,15,0,-,2021-07-07 08:09:16,0,0,1970-01-01 00:00:00,peloton,...,134,4091,| Date | Stage | From > To | Length | Type | F...,2021-07-07 08:00:20,0,477,0,0,1,Hard
4,anonymous,A couple of links ahead of the stage:\n\n* If ...,14,0,-,2021-07-07 08:09:16,0,0,1970-01-01 00:00:00,peloton,...,134,4091,| Date | Stage | From > To | Length | Type | F...,2021-07-07 08:00:20,0,477,0,0,1,Hard


8 - Sentiment Analysis [6 marks]

Using SpaCy, perform sentiment analysis on the `comment_body` column of your dataframe `df` [2.5 marks] and create the following new columns:

a - `positive`: the positive score of the sentiment analysis [1 mark]

b - `negative`: the negative score of the sentiment analysis [1 mark]

c - `sentiment`: the negative score subtracted from the positive score [0.5 mark]

Create a new dataframe `df_sent` which includes the following 2 columns: `comment_body`, `sentiment` and print the `df_sent` dataframe to a `.csv` file named `comment_sentiment.csv` [1 mark].


In [18]:
# your code here
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl (13.7 MB)


In [19]:
!pip install vaderSentiment

  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)


In [20]:
import spacy
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [21]:
analyzer = SentimentIntensityAnalyzer()

In [22]:
def get_sentiment(text):
    """This function receives a text and returns its sentiment polarity and subjectivity."""
    doc = nlp(text)
    vader_score = analyzer.polarity_scores(text)
    lis = [vader_score['neg'],vader_score['neu'],vader_score['pos'],vader_score['compound']]
    return lis

In [23]:
# df['Polarity'], df['Subjectivity'] = zip(*df['comment_body'].map(get_sentiment))
# df['Sentiment'] = df['Polarity'].apply(lambda polarity : 'Positive' if polarity > 0 else 'Negative' if polarity < 0 else 'Neutral')
df['negative'], df['neu'],df['positive'],df['com'] = zip(*df['comment_body'].map(get_sentiment))

In [24]:
df['sentiment'] = df['negative'] - df['positive']

In [25]:
df.head()

,author,comment_body,comment_score,comment_award,comment_mod_distinction,comment_created,author_comment_karma,author_post_karma,author_created,subreddit,...,comment_length,author_total_karma,higher_post_karma,stage_nbr,stage_type,negative,neu,positive,com,sentiment
0,arne-b,Campenaerts actually prepared pretty hard for ...,106,0,-,2021-06-26 10:47:04,54881,11172,2014-01-26 16:34:12,peloton,...,129,66053,0,1,Medium,0.053,0.755,0.192,0.5719,-0.139
1,Noble_Flatulence,"The bikewrench subreddit be like ""frame slight...",78,0,-,2021-06-26 14:15:01,181898,4853,2013-01-10 03:29:48,peloton,...,86,186751,0,1,Medium,0.000,0.690,0.310,0.6597,-0.310
2,tkcom,That fan should be fined based on the damage c...,79,0,-,2021-06-26 14:15:07,84779,3709,2010-11-05 18:37:48,peloton,...,56,88488,0,1,Medium,0.335,0.516,0.148,-0.4404,0.187
3,Cog348,Chris Froome getting cheered by a French crowd...,77,0,-,2021-06-26 15:17:15,93846,1990,2019-01-14 21:43:15,peloton,...,151,95836,0,1,Medium,0.000,0.723,0.277,0.8615,-0.277
4,Sead_KolaSagan,Thomas escaped the crash that took down everyb...,67,0,-,2021-06-26 14:15:22,32150,88,2017-09-23 12:08:23,peloton,...,73,32238,0,1,Medium,0.168,0.621,0.211,0.1779,-0.043


In [26]:
df_sent = df[['comment_body','sentiment']]

In [27]:
df_sent.head()

,comment_body,sentiment
0,Campenaerts actually prepared pretty hard for ...,-0.139
1,"The bikewrench subreddit be like ""frame slight...",-0.310
2,That fan should be fined based on the damage c...,0.187
3,Chris Froome getting cheered by a French crowd...,-0.277
4,Thomas escaped the crash that took down everyb...,-0.043


9 - Statistical Analysis [6 marks]

Using SciPy, evaluate the following null hypotheses:

a - The `comment_score` of comments on the post of stage 8 and stage 11 is the same

b - There is no correlation between the `comment_score` and the `comment_length` of comments about stage 11 posted on the day of the stage (i.e. comments that are included in the dataframe s11).

For each of the two hypotheses:

i - Check and explain with comments what statistical test is suitable for each hypothesis [1 mark per hypothesis]

ii - Compute and print the test statistics and p-value [1 mark per hypothesis]

iii - Write in a comment whether the hypothesis should be rejected or not and what conclusions you can draw from this. You may need to compute additional descriptive statistics for this part. [1 mark per hypothesis]


Hypothesis a

Statistical Test: To compare the comment_score of comments on the post of stage 8 and stage 11, a suitable test would be the independent two-sample t-test. This test compares the means of two independent groups to determine if they are significantly different from each other.

In [28]:
# your code here
from scipy import stats

df_stage_8 = df[df.post_title.str.contains('Stage 8')]

# Assuming comment_scores_stage8 and comment_scores_stage11 are arrays of comment scores for stages 8 and 11 respectively
t_statistic, p_value = stats.ttest_ind(df_stage_8['comment_score'], df_stage_11['comment_score'])
print("Test Statistic:", t_statistic)
print("p-value:", p_value)

Test Statistic: 2.201253352891754
p-value: 0.02788125864675384


Conclusion: If the p-value is less than the chosen significance level (commonly 0.05), we reject the null hypothesis. This would imply that there is a significant difference in the comment_score between stage 8 and stage 11. Otherwise, we fail to reject the null hypothesis, suggesting that there is no significant difference in the comment_score between the two stages.

Hypothesis B

Statistical Test: To test for correlation between comment_score and comment_length for comments about stage 11 posted on the day of the stage, we can use Pearson's correlation coefficient. This test measures the strength and direction of the linear relationship between two variables.

In [29]:
# Assuming comment_scores_s11 and comment_lengths_s11 are arrays of comment scores and lengths for stage 11 respectively
correlation_coefficient, p_value = stats.pearsonr(s11['comment_score'], s11['comment_length'])
print("Correlation Coefficient:", correlation_coefficient)
print("p-value:", p_value)

Correlation Coefficient: 0.10190954364356114
p-value: 0.004781183377557273


Conclusion: If the p-value is less than the chosen significance level (e.g., 0.05), we reject the null hypothesis, indicating that there is a significant correlation between comment_score and comment_length. Otherwise, if the p-value is greater than the significance level, we fail to reject the null hypothesis, suggesting that there is no significant correlation between the two variables.